In [ ]:
# 查看各种语句的几个例子

object_func_call  = set()
folder_path = '.\output\\'
file_list = read_dir(folder_path)
function_list = set()
for filename in file_list:
    with open(folder_path + filename, 'r') as f:
        for line in f:
            if is_func_call(line):
                res = parse_func_call(line)
                function_list.add(res[0])
                

In [1]:
from build_graph import *
from write_new_file import *
from tool_function import *
graph = Graph()
nodelist = NodeList()
mapstack = MapStack()
folder_path = '.\output\\'
file_list = read_dir(folder_path)
for filename in file_list:
    read_to_graph(folder_path+filename,graph,nodelist,mapstack)
# graph.print_edges()

node = Node("pBuf_13",[])
print(graph.has_vertex(node))

True


In [2]:
res = []
graph.dfs("blFactor_1",res)
for i in  res:
    print(i)

['.\\output\\3DMarkICFWorkload_32.sdx', 537]


In [ ]:
# 现在是做帧的简化步骤，要求是从第i帧到第j帧，从某个文件开始到某个文件结束，比如截取58到60帧，从文件序号59开始，到文件序号61结束
# 然后组成新的文件

# 1. 读取文件夹下的所有文件，然后按照文件名排序，然后按照文件名的数字排序
# 2. 先用所有的文件构成一个图，

# 建图后该怎么进行重构，
# 先试着截取1帧
new_file_list = [] # 保存文件行所在位置
filename = ".\output\\3DMarkICFWorkload_58.sdx"
with open(filename, 'r') as f:
    line_pos = 0
    for line in f:
        if is_comment(line):
            pass
        elif is_empty(line):
            pass
        elif is_other(line):
            new_file_list.append([filename,line_pos])
            res = parse_statement(line)
            p_v = map(str.strip, res[3].split(","))
            for i in p_v:
                # 第一个是类型，第二个是变量名称，第三个是数组，第四个才是依赖
                if( is_decimal_or_hex(i)):
                    pass
                elif(is_macro(i)):
                    pass
                elif(is_valid_variable(i)):
                    # 当是合法的变量时候，应该去以res[i]去深度遍历所有依赖的节点。
                    dependency_node = [] 
                    graph.dfs(i,dependency_node)
                    new_file_list.extend(dependency_node)
                else:
                    pass
        elif is_object_func_call(line):
            new_file_list.append([filename,line_pos])
            res = parse_object_func_call(line)
            # 对象->函数(参数)
            for i in res[2]:
                p_v = parse_func_assignment(i)
                if(is_macro(p_v[1])):
                    pass
                elif(is_decimal_or_hex(p_v[1])):
                    pass
                elif(is_valid_variable(p_v[1])):
                    dependency_node = [] 
                    graph.dfs(p_v[1],dependency_node)
                    new_file_list.extend(dependency_node)
        elif is_func_call(line):
            new_file_list.append([filename,line_pos])
        # 如果这行引用到了其他文件的变量，则去graph寻找
        
        line_pos = line_pos+1

    # 最后根据new_file_list中的行号和文件序号，重新组成一个文件

In [ ]:
create_new_sdx_file(new_file_list,"./test-script/3DMarkICFWorkload_1.sdx")